In [2]:
time1=time()
for i in range(30):
    self = Scoring(PDB_file1, PDB_file2, max_aip_dist=0.15, branch=True, solvent="water")
time2=time()
print(time2-time1)

NameError: name 'time' is not defined

In [4]:
from ssipscore.asc import calculate_association_energy as ac
from ssipscore.PDB_preprocess import PDB_preprocess 
from ssipscore.getScoring import Scoring
from ssipscore.create_jmol_vis import create_jmol_vis, create_ip_vis
from ssipscore.get_desolvation import get_desolvated_AIPs
from ssipscore.get_desolvation import get_free_energy_of_solvation
from ssipscore.get_desolvation import get_free_energy_of_phase_transfer
from time import time
import mdtraj as md
import pandas as pd
import numpy as np
import os
X = "NMe2"
path = "/home/kate/workspace/Calixpyrrole"
PDB_file2 = f"{path}/host/host.pdb"
PDB_file1 = f"{path}/guest{X}/guest{X}.pdb"
PDB_preprocess(PDB_file1, PDB_file2)

In [7]:
#self = Scoring(PDB_file1, PDB_file2, max_aip_dist=0.17, branch=False, solvent="chloroform")
self = Scoring(PDB_file1, PDB_file2, max_aip_dist=0.12, branch=False, solvent="water")

In [8]:
self.interaction_df

,L,R,Atom_Distance,L_AIP,R_AIP,AIP_Distance,L_frac,R_frac,L_type,R_type,L_value,R_value,ddG,ddG_sc
0,10.0,132.0,0.320372,52.0,382.0,0.028126,0.5,0.5,C.ar,C.ar,-1.75,-1.88,-1.682278,-0.841139
1,21.0,147.0,0.265975,64.0,237.0,0.054332,1.0,0.5,H.soft,C.ar,1.12,-1.85,-2.706267,-1.353133
2,27.0,138.0,0.237657,70.0,288.0,0.090356,0.5,0.5,H.soft,C.ar,0.94,-1.46,-3.205439,-1.602720
3,27.0,177.0,0.163638,70.0,348.0,0.061063,0.5,1.0,H.soft,H.soft,0.94,0.94,-2.492416,-1.246208
4,13.0,130.0,0.308543,57.0,387.0,0.063789,0.5,0.5,N.pl3.aniline,C.ar,-0.84,-1.47,-3.197828,-1.598914
5,7.0,139.0,0.320914,45.0,286.0,0.069256,0.5,0.5,C.ar,C.ar,-1.39,-1.85,-2.185473,-1.092736
6,9.0,135.0,0.330433,49.0,283.0,0.071651,0.5,0.5,C.ar,C.ar,-1.74,-1.77,-1.857236,-0.928618
7,19.0,122.0,0.277774,62.0,338.0,0.104742,0.5,0.5,H.soft,C.ar,1.10,-1.47,-2.998132,-1.499066
8,19.0,123.0,0.249821,62.0,340.0,0.082689,0.5,0.5,H.soft,C.ar,1.10,-1.74,-2.816280,-1.408140
9,9.0,128.0,0.348657,50.0,391.0,0.099636,0.5,0.5,C.ar,C.ar,-1.68,-1.79,-1.911376,-0.955688


In [6]:
def get_sasa_per_AIP(self, traj, probe_radius, L=True, only=True):
    """wrapper for shrake_rupley to also quantify sasa changed per AIP"""
    if L == True:
        Atom = self.Atom_L
        AIP = self.AIP_L
    else: 
        Atom = self.Atom_R
        AIP = self.AIP_R
    if L == False and only == True:
        L = "Ronly"
        
    y, a = md.shrake_rupley(traj, probe_radius=probe_radius, n_sphere_points=1000, get_mapping=False)
    xdim = int(len(a[0])/3000)
    b = a.reshape(xdim,1000,3)
        
    for atom_i in Atom.keys():   
        aip_ind = np.array([i.index for i in AIP.values() if (i.atom == atom_i)])
        if len(aip_ind) == 0:
            continue
            
        atom_ii = atom_i
        if L == False:
            atom_i = atom_i - len(self.AIP_L)
        if L == "Ronly":
            atom_i = atom_i - len(self.AIP_L) - len(self.Atom_L)
            
        area_atom_i = y[0][atom_i]

        if area_atom_i > 0:
            aip_xyz = np.array([i.xyz for i in AIP.values() if (i.atom == atom_ii)])
            data = b[atom_i]
            data = data[~np.all(data == 0, axis=1)]
            JJ = np.array([np.linalg.norm(data-a, axis=1) for a in aip_xyz]).T
            jj = aip_ind[JJ.argmin(axis=1)]
            no_jj = len(jj) / area_atom_i
            for k in aip_ind:
                if only == False:
                    AIP[k].set_sasa_b(np.count_nonzero(jj == k) / no_jj)
                else:
                    AIP[k].set_sasa_f(np.count_nonzero(jj == k) / no_jj)
        else:
            for k in aip_ind:
                if only == False:
                    AIP[k].set_sasa_b(0.0)
                else:
                    AIP[k].set_sasa_f(0.0)
    return y

In [9]:
    all_atoms = list(self.state.ligand_atom_indices)+list(self.state.residue_atom_indices)
    traj = self.mdTrajectory.atom_slice(all_atoms)
    yy = get_sasa_per_AIP(self, traj, probe_radius=0.2, L=True, only=False)
    yy = get_sasa_per_AIP(self, traj, probe_radius=0.2, L=False, only=False)
    Ltraj = self.mdTrajectory.atom_slice(list(self.state.ligand_atom_indices))
    Rtraj = self.mdTrajectory.atom_slice(list(self.state.residue_atom_indices))
    xx = get_sasa_per_AIP(self, Ltraj, probe_radius=0.2, L=True, only=True)
    xx = get_sasa_per_AIP(self, Rtraj, probe_radius=0.2, L=False, only=True)
    [(i.value, i.type, round(i.sasa_f - i.sasa_b, 4)) for i in self.AIP_L.values()]

[(-0.71, 'C.ar', 0.1703),
 (-0.86, 'C.ar', 0.0),
 (0.19, 'C.ar', 0.1118),
 (0.1, 'C.ar', 0.0),
 (0.19, 'C.ar', 0.0464),
 (0.16, 'C.ar', 0.0),
 (0.11, 'C.ar', 0.117),
 (0.15, 'C.ar', 0.0),
 (-0.92, 'C.ar', 0.1738),
 (-0.96, 'C.ar', 0.0),
 (-8.49, 'O.2.noxide', 0.6539),
 (-6.75, 'O.2.noxide', 0.0),
 (-6.56, 'O.2.noxide', 0.0),
 (-1.03, 'N.ar.no_lp', 0.0206),
 (-1.1, 'N.ar.no_lp', 0.0),
 (-0.44, 'C.ar', 0.0275),
 (-0.42, 'C.ar', 0.0258),
 (-1.22, 'C.ar', 0.0688),
 (-1.17, 'C.ar', 0.0637),
 (-0.5, 'C.ar', 0.043),
 (-0.6, 'C.ar', 0.0688),
 (-0.4, 'C.ar', 0.0464),
 (-0.33, 'C.ar', 0.0499),
 (-0.41, 'C.ar', 0.0688),
 (-0.32, 'C.ar', 0.0482),
 (-0.26, 'C.ar', 0.0206),
 (-0.29, 'C.ar', 0.0241),
 (-0.87, 'C.2', 0.0447),
 (-0.91, 'C.2', 0.0344),
 (-4.74, 'O.2.carbonyl', 0.0841),
 (-5.27, 'O.2.carbonyl', 0.1401),
 (-1.29, 'O.2.carbonyl', 0.19),
 (-1.29, 'O.2.carbonyl', 0.1837),
 (1.49, 'H.soft', 0.323),
 (1.57, 'H.soft', 0.1956),
 (1.45, 'H.soft', 0.3256),
 (0.9, 'H.soft', 0.2162),
 (1.31, 'H.soft

In [171]:
create_ip_vis(self, f"guest{X}_host")

In [187]:
L = self.interaction_df.L_value
R = self.interaction_df.R_value

In [188]:
for ind, i in enumerate(L):
    j = R[ind]
    print(-get_free_energy_of_solvation(i, "chloroform"), -get_free_energy_of_solvation(j, "chloroform"))

1.272636229692969 2.0054516778233857
1.3040357626171504 2.540829203369439
1.1915665476853876 1.7830200966726846
1.3058002113493223 1.964093494246047
1.495759359778173 1.3837170444104336
1.4775462890234434 1.9875454809408188
1.495759359778173 2.0054516778233857
1.302273437517825 1.3855774559687735
1.4775462890234434 1.9875454809408188
1.2709116701823457 1.9816374222274662
1.3164316480554612 1.3800029187103962
1.3040357626171504 1.9130528695134141
1.2865076347163134 1.9411172845295253
1.4716429019062034 1.7830200966726846
1.2623199008591979 1.3855774559687735
1.488234851376497 1.9241921683376413
1.2847663988381024 1.9298049090777338
1.2865076347163134 1.964093494246047
1.2865076347163134 1.9525464483938553
1.2709116701823457 1.9411172845295253
1.4716429019062034 1.9241921683376413
2.851473009018689 1.3837170444104336
1.3080376913406453 1.3837170444104336
1.3058002113493223 1.9875454809408188
1.2760915763710305 1.7781532250390892
1.511396988342219 1.9525464483938553
1.272636229692969 1.77

In [189]:
for ind, i in enumerate(L):
    j = R[ind]
    print(-get_free_energy_of_solvation(i, "water"), -get_free_energy_of_solvation(j, "water"))

-0.6487271579948607 0.6651113286727766
-0.48805343736241025 1.9575290275012414
-0.7750158257240651 0.1107500910079021
-0.4776280096405002 0.5617082889015308
-0.518716136204127 0.14658660461597658
1.3013759310603759 0.6203671547794757
-0.518716136204127 0.6651113286727766
-0.4983103878343189 0.1654296433599698
1.3013759310603759 0.6203671547794757
-0.656289044732498 0.6055949349722276
-0.4114496349265977 0.10950465419816939
-0.48805343736241025 0.4339593996329427
-0.5833349300786429 0.504202698236138
1.2176123552310172 0.1107500910079021
-0.692241947985513 0.1654296433599698
-0.5315011303336696 0.461837032223244
-0.5920077810505228 0.47588586118862564
-0.5833349300786429 0.5617082889015308
-0.5833349300786429 0.532810634447224
-0.656289044732498 0.504202698236138
1.2176123552310172 0.461837032223244
2.658977924359605 0.14658660461597658
-0.7393284859590447 0.14658660461597658
-0.4776280096405002 0.6203671547794757
-0.633210330954818 0.09881332037204961
1.8054270448296441 0.5328106344472

In [190]:
for ind, i in enumerate(L):
    j = R[ind]
    print(get_free_energy_of_phase_transfer(i, "chloroform", "hexadecane"), get_free_energy_of_phase_transfer(j, "chloroform", "hexadecane"))

0.0107091122047831 0.4902997535107556
0.039549716839923255 0.9155026812156488
-0.06623628153632999 0.32680388162970164
0.04117025635669447 0.4591087630698798
0.13409778304450914 0.1127407564430869
0.19902441771443113 0.476756256840557
0.13409778304450914 0.4902997535107556
0.037931121185027505 0.11445019202182216
0.19902441771443113 0.476756256840557
0.009124926238328612 0.47230065337445115
0.05093450600353844 0.10932815862304124
0.039549716839923255 0.42107640961146986
0.02345067180027649 0.44192285559693234
0.1935907466845681 0.32680388162970164
0.001232127559172902 0.11445019202182216
0.1292959250260315 0.42933117188269954
0.021851327549111677 0.4335003193148499
0.02345067180027649 0.4591087630698798
0.02345067180027649 0.45045864028017957
0.009124926238328612 0.44192285559693234
0.1935907466845681 0.42933117188269954
1.1738554479386032 0.1127407564430869
0.0134876336150338 0.1127407564430869
0.04117025635669447 0.476756256840557
0.013883136791450923 0.32335606869567957
0.2301974382

In [191]:
for ind, i in enumerate(L):
    j = R[ind]
    print(get_free_energy_of_phase_transfer(i, "water", "hexadecane"), get_free_energy_of_phase_transfer(j, "water", "hexadecane"))

-1.9106542754830467 -0.8500405956398536
-1.7525394831396373 0.3322025053474511
-2.0328186549457827 -1.345466124035081
-1.742257964633128 -0.9432764422746365
-1.880377712937791 -1.1243896833513702
0.022854059751363565 -0.8904220693207862
-1.880377712937791 -0.8500405956398536
-1.7626527041671165 -1.1056976205869815
0.022854059751363565 -0.8904220693207862
-1.9180757886765152 -0.9037418338807874
-1.6769467769785205 -1.1611701058891855
-1.7525394831396373 -1.0580170602690016
-1.8463918929946797 -0.994991730696455
-0.060439799990618104 -1.345466124035081
-1.953329721285538 -1.1056976205869815
-1.890440056684135 -1.0330239642316976
-1.8549228523395134 -1.0204187285742583
-1.8463918929946797 -0.9432764422746365
-1.8463918929946797 -0.9692771736664517
-1.9180757886765152 -0.994991730696455
-0.060439799990618104 -1.0330239642316976
0.9813603632795194 -1.1243896833513702
-2.033878543684656 -1.1243896833513702
-1.742257964633128 -0.8904220693207862
-1.8954187705343977 -1.35598383597136
0.5242274

In [192]:
L = [l.value for _, l in self.AIP_L.items() if l.xyz[0] < 10]
Lf = [l.fraction for _, l in self.AIP_L.items() if l.xyz[0] < 10]
La = [l.type for _, l in self.AIP_L.items() if l.xyz[0] < 10]

In [193]:
ll = []; ww = []; qq = []; ee = []
for ind, i in enumerate(L):
    ll.append(-get_free_energy_of_solvation(i, "chloroform")*Lf[ind])
    ww.append(-get_free_energy_of_solvation(i, "water")*Lf[ind])
    qq.append(get_free_energy_of_phase_transfer(i, "chloroform", "hexadecane")*Lf[ind])
    ee.append(get_free_energy_of_phase_transfer(i, "water", "hexadecane")*Lf[ind])    

In [194]:
print(sum(ll), sum(ww), sum(qq), sum(ee))

26.193374995628147 5.038202455435812 3.620054412575086 -17.535118127617253


In [18]:
poly_coeffs_wat = {'positive': np.array([0.8419999699999999, -0.519020007, -7.95083649E-02, -8.12364268E-01, 3.524312E-01, -7.050761775E-02, 7.61994779E-03, -4.2963367E-04, 9.92816837E-06]),
                   'negative': np.array([0.778951615, 1.26502296E-02, -3.731657205E-01, 4.08016045E-02, 1.72853731E-02, 2.092557689E-03, 1.25465594E-04, 3.79343575E-06, 4.6163579E-08])}
poly_coeffs_hex = {'positive': np.array([-1.24045027, 1.89024615E-14, -2.32155075E-14, 1.36380230E-14, -4.43948954E-015, 8.406146166E-016, -9.203165934E-17, 5.39271494E-18, -1.36621071E-19]),
                   'negative': np.array([-1.23921832, 0.29382657, -9.8692988E-03, 2.59832152E-04, 7.9892149E-05, 6.69220165E-06, 3.1605919E-07, 8.2558549E-09, 9.18654608E-11])}
poly_coeffs_chl = {'positive': np.array([-1.220126, -1.662607E-01, -9.94286E-03, -4.672309E-04, -2.678255E-06, 1.475957E-05, -2.411077E-06, 1.74012E-07, -4.822481E-09]),
                   'negative': np.array([-1.187073, 4.507678E-01, 1.42246E-01, 9.4427E-02, 1.417198E-02, 1.072633E-03, 4.489395E-05, 9.877984E-07, 8.891901E-09])}
theta_hex = 0.4534082818811403
theta_wat = 0.7384892873546126
theta = 0.4266047258483052

poly = poly_coeffs_hex
t = theta_hex

EvdW = -5.6  # kJmol^-1.
GAS_CONSTANT = 8.3144598  # Jmol^-1.
temperature=298.0

In [19]:
[print(b, Lf[i],La[i]) for i, b in enumerate(ll)];

0.8723746449654466 0.5 C.ar
0.8866552313779649 0.5 C.ar
0.9482785977841327 0.5 C.ar
0.959304101553123 0.5 C.ar
0.9620960841688206 0.5 C.ar
0.9455574586882988 0.5 C.ar
0.9401569354697078 0.5 C.ar
0.9649024545388669 0.5 C.ar
0.9649024545388669 0.5 C.ar
0.9401569354697078 0.5 C.ar
0.9348115690814873 0.5 C.ar
0.9348115690814873 0.5 C.ar
0.7573468590556687 0.5 N.pl3.aniline
0.6637085963278636 0.5 N.pl3.aniline
1.4156496083332024 1.0 H.soft
1.4005414190138934 1.0 H.soft
1.4194493225322025 1.0 H.soft
1.404304900894517 1.0 H.soft
1.4540582830482938 1.0 H.soft
1.3986630595604481 1.0 H.soft
1.3874401030678258 1.0 H.soft
1.3855774559687735 1.0 H.soft
1.3949130925183926 1.0 H.soft
1.4560028575490727 1.0 H.soft


In [20]:
L = self.interaction_df.L_value
R = self.interaction_df.R_value

In [21]:
bind = []
for ind, i in enumerate(L):
    j = R[ind]

    K_vdW = 0.5 * np.exp(-EvdW * np.power(10, 3) / (GAS_CONSTANT * temperature))

    interaction_energy = i * j #if i * j < 0.0 else 0.0
    K = 0.5 * np.exp(-(interaction_energy + EvdW) * np.power(10, 3) / (GAS_CONSTANT * temperature))

    association_constant = K + K_vdW
    frac_num = -1.0 + np.sqrt(1.0 + 4 * association_constant * t)
    frac_den = 2 * association_constant * t
    binding_energy = 2 * GAS_CONSTANT * temperature * np.log(frac_num / frac_den) / np.power(10, 3)
    c_frac_num = -1.0 + np.sqrt(1.0 + 8 * t)
    c_frac_den = 4 * t
    dG_confinement = - 2 * GAS_CONSTANT * temperature * np.log(c_frac_num / c_frac_den) / np.power(10, 3)
    #ii = (polynom.polyval(i, poly["negative"]) if i < 0 else polynom.polyval(i, poly["positive"]))
    #jj = (polynom.polyval(j, poly["negative"]) if j < 0 else polynom.polyval(j, poly["positive"]))
    bind.append(binding_energy+dG_confinement)

In [22]:
[print(b) for b in bind];

-3.4555781896218063
-1.735807464084238
-1.7252529190971706
-1.8361381111384363
-2.2489929838504583
-1.5983039511458381
-2.156604097998179
-3.5529513203427285
-3.5529513203427285
-1.7665208022931385
-3.183380913584201
-2.265779218798408
-1.7605370578335062
-3.415326873639093
-1.7577002413672052
-3.359228518413154
-1.8460381713558132
-3.302807015367197
-1.7864515942188897
-3.175960922479856
-3.98699258040564
-3.4640183821730783
-1.7639728554912626
-1.8008972284365141
-2.2454882050050173
-2.259879596615117
-1.7808635258238326
-2.6905809380340067
-2.194973106298015
-4.0216645852187245
-4.0216645852187245


In [ ]:
self = Scoring(PDB_file1, PDB_file2, max_aip_dist=0.17, branch=False, solvent="chloroform")
try:
    dG1 = self.H_bond_df.ddG.sum()
    dG2 = self.non_polar_df.ddG_sc.sum()
    fracx = self.non_polar_df.L_frac * self.non_polar_df.R_frac
    frac = np.array([1 if x == 1 else 0.5 for x in fracx])
    dG3 = sum(self.H_bond_df.L_value*self.H_bond_df.R_value)
    dG4 = sum([i for i in self.non_polar_df.L_value*self.non_polar_df.R_value*frac])
    print(dG1, dG2, dG3, dG4)
except:
    pass

create_ip_vis(self, f"guest{X}_host")
#self.interaction_df.to_csv(f"{comps[0]}_{comps[1]}.csv")
self.desolv_energy
self.transfer_energy
self.non_polar_add

In [28]:
[print(b, L[i], R[i]) for i, b in enumerate(bind)];

-2.333566633605952 -0.36 -1.81
-2.2656863251807016 -0.47 -1.84
-2.259879596615117 -0.47 -1.88
-2.3154376424203624 -0.27 -2.62
-2.0047677159936335 -1.24 -1.47
-3.7919605174523543 1.34 -1.85
-3.7919605174523543 1.34 -1.85
-2.3607468939434435 -0.39 -1.46
-2.3962949818101844 -0.27 -1.72
-3.5034306884127773 1.35 -1.47
-2.281458204938909 -0.46 -1.77
-3.7974650501769442 1.39 -1.79
-4.6658372347990325 1.39 -2.74
-2.328863357138906 -0.36 -1.85
-3.066881526640029 -1.24 0.93
-2.327657669334375 -0.37 -1.81
-2.3184411535183758 -0.39 -1.79
-2.327038975755939 -0.46 -1.46
-3.7140844145042458 1.35 -1.74
-4.685229279421168 1.4 -2.74
-4.685229279421168 1.4 -2.74
-2.332487270927852 -0.37 -1.77
-2.883343878362436 -0.82 0.93
-2.878556830825578 -0.8 0.94


In [92]:
    i = 1
    j = 1
    K_vdW = 0.5 * np.exp(-EvdW * np.power(10, 3) / (GAS_CONSTANT * temperature))

    interaction_energy = i * j #if i * j < 0.0 else 0.0
    K = 0.5 * np.exp(-(interaction_energy + EvdW) * np.power(10, 3) / (GAS_CONSTANT * temperature))

    association_constant = K + K_vdW
    frac_num = -1.0 + np.sqrt(1.0 + 4 * association_constant * t)
    frac_den = 2 * association_constant * t
    binding_energy = 2 * GAS_CONSTANT * temperature * np.log(frac_num / frac_den) / np.power(10, 3)
    
    
    c_frac_num = -1.0 + np.sqrt(1.0 + 8 * t)
    c_frac_den = 4 * t
    dG_confinement = - 2 * GAS_CONSTANT * temperature * np.log(c_frac_num / c_frac_den) / np.power(10, 3)
    
    bind = binding_energy+dG_containment

In [167]:
create_ip_vis(self, f"guest{X}_host")

In [58]:
get_free_energy_of_phase_transfer(1, "water", "hexadecane")

-0.560681424638364